In [1]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp


In [2]:
project_name = 'apache-log4j'
project_releases = pd.read_csv('project_links_202.csv')

In [3]:
project_releases.head()

,project_name,project_link,revision,version_name
0,ZieIony-Carbon,https://github.com/ZieIony/Carbon,567fd84,7.1.84
1,ZieIony-Carbon,https://github.com/ZieIony/Carbon,9a343f5,7.1.83
2,ZieIony-Carbon,https://github.com/ZieIony/Carbon,a655476,7.1.82
3,ZieIony-Carbon,https://github.com/ZieIony/Carbon,ad84a93,7.1.81
4,ZieIony-Carbon,https://github.com/ZieIony/Carbon,5fa652f,7.1.80


In [4]:
current_project = pd.DataFrame(project_releases[project_releases['project_name'] == project_name])
current_project['rank'] = current_project['version_name'].rank()
current_project = current_project.sort_values(by=['rank'], ascending=False)
current_project.head(30)

,project_name,project_link,revision,version_name,rank
1071,apache-log4j,https://github.com/apache/log4j,31d7ec7,8.2.0,21.0
1072,apache-log4j,https://github.com/apache/log4j,fcbe46c,8.1.1,20.0
1073,apache-log4j,https://github.com/apache/log4j,dbe5ed0,8.1.0,19.0
1074,apache-log4j,https://github.com/apache/log4j,2ae4746,8.0.0,18.0
1075,apache-log4j,https://github.com/apache/log4j,d4c30fc,7.7.2,17.0
1076,apache-log4j,https://github.com/apache/log4j,5bf96d3,7.7.1,16.0
1077,apache-log4j,https://github.com/apache/log4j,8c831da,7.7.0,15.0
1078,apache-log4j,https://github.com/apache/log4j,719cde9,7.6.0,14.0
1079,apache-log4j,https://github.com/apache/log4j,b5bf70b,7.5.0,13.0
1080,apache-log4j,https://github.com/apache/log4j,9060ac6,7.4.0,12.0


In [ ]:
### Download dataset
try:
    os.mkdir('raw_sourcecode/' + project_name)
except:
    pass
for row in current_project.iterrows():
    command = 'cd C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/' + project_name +' & mkdir ' + project_name + '_' +row[1]['version_name']
    
    #print(command)
    os.system(command)
    
    command = 'git clone ' + row[1]['project_link'] +  ' C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/' + project_name + '/' + project_name + '_' + row[1]['version_name']
    
    print(command)
    os.system(command)
    command = 'cd C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/' + project_name + '/' + project_name +'_' + row[1]['version_name'] + ' & git checkout ' + row[1]['revision']
    print(command)
    os.system(command)

In [10]:
### Run Depends
try:
    os.mkdir('raw_depends/' + project_name)
except:
    pass
for row in current_project.iterrows():
    #command = 'cd C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/' + project_name +' & mkdir ' + project_name + '_' +row[1]['version_name']
    #os.system(command)
    #print(command)
    command = 'cd C:/Users/tanji/Desktop/SoftwareRemodularization/depends-0.9.2 & ' + 'java -jar depends.jar java C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/'  + project_name + '/' + project_name + '_' + row[1]['version_name']+  ' ../raw_depends/' + project_name + "/" + project_name + '_' + row[1]['version_name'] 
    print(command)
    os.system(command)

cd C:/Users/tanji/Desktop/SoftwareRemodularization/depends-0.9.2 & java -jar depends.jar java C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/apache-log4j/apache-log4j_8.2.0 ../raw_depends/apache-log4j/apache-log4j_8.2.0
cd C:/Users/tanji/Desktop/SoftwareRemodularization/depends-0.9.2 & java -jar depends.jar java C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/apache-log4j/apache-log4j_8.1.1 ../raw_depends/apache-log4j/apache-log4j_8.1.1
cd C:/Users/tanji/Desktop/SoftwareRemodularization/depends-0.9.2 & java -jar depends.jar java C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/apache-log4j/apache-log4j_8.1.0 ../raw_depends/apache-log4j/apache-log4j_8.1.0
cd C:/Users/tanji/Desktop/SoftwareRemodularization/depends-0.9.2 & java -jar depends.jar java C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/apache-log4j/apache-log4j_8.0.0 ../raw_depends/apache-log4j/apache-log4j_8.0.0
cd C:/Users/tanji/Desktop/SoftwareRemodularization/depen

In [13]:
try:
    os.mkdir('groundtruth/' + project_name)
except:
    pass

try:
    os.mkdir('MoJo_1.2.1/' + project_name)
except:
    pass


### Counter to only ensure the top 10 projects are selected
counter = 10

for row in current_project.iterrows():
    counter -= 1
    if counter < 0:
        break
    project_rank = row[1]['rank']
    version_name = row[1]['version_name']
    #print(project_rank)
    rootdir = 'C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/' + project_name + '/' + project_name + '_' + row[1]['version_name']
    
    #print(rootdir)
    
    
    ### To obtain the current initial directory
    full_dir_arr = []
    for root, dirs, files in os.walk(rootdir):
        #print(root)
        #print(dirs)
        for element in files:
            if '.java' in element:
                dir_string = root + '\\' + element
                full_dir_arr.append(dir_string)

    cluster_dict = {}
    cluster_tree = {}


    for element in full_dir_arr:
        element = element.split('\\')
        child = element[-1]
        parent = element[-2]
        cluster_tree[child] = parent
            
    current_rank = project_rank
    while current_rank >= (project_rank - 10):
        current_rank -= 1
        project_to_be_compared = current_project[current_project['rank'] == current_rank]
        project_to_be_compared_rootdir = 'C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/' + project_name + '/' + project_name + '_' + project_to_be_compared['version_name'].values[0]
        #print(project_to_be_compared_rootdir)


        ### To obtain the current initial directory
        project_to_be_compared_full_dir_arr = []
        for root, dirs, files in os.walk(project_to_be_compared_rootdir):
            #print(root)
            #print(dirs)
            for element in files:
                if '.java' in element:
                    dir_string = root + '\\' + element
                    project_to_be_compared_full_dir_arr.append(dir_string)

        project_to_be_compared_cluster_dict = {}
        project_to_be_compared_cluster_tree = {}


        for element in project_to_be_compared_full_dir_arr:
            element = element.split('\\')
            child = element[-1]
            parent = element[-2]
            project_to_be_compared_cluster_tree[child] = parent
                
        cluster_tree =  {x:cluster_tree[x] for x in cluster_tree if x in project_to_be_compared_cluster_tree} 
    
    arr_a_rsf = []
    filename = 'C:/Users/tanji/Desktop/SoftwareRemodularization/groundtruth/' + project_name + '/' + project_name + '_' + row[1]['version_name'] + '.txt'
    with open(filename, 'w') as f:
        for key, value in cluster_tree.items():
            arr_a_rsf.append(key)
            f.write('contain ' + str(value).replace(" ",'') + ' ' + str(key).replace(' ','') + '\n')
        
    depends_dir = 'C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/' + project_name + '/' + project_name + '_' + row[1]['version_name'] + '.json'
    print(depends_dir)

    with open(depends_dir) as f:
        depends_results = json.load(f)
        index  = 0
        var_array = []
        for value in depends_results['variables']:
            var_array.append([index, value.split('\\')[-1]])
            #print(index, value)
            index += 1

        var_df = pd.DataFrame(var_array)
        var_df.columns = ['index_val', 'name']

        feature_list = {}
        feature_index = 2
        for element in depends_results['cells']:
            #print(element)
            try:
                for a in element['values']:
                    if a not in feature_list:
                        feature_list[a] = feature_index
                        feature_index += 1
                    #print(a['Call'])


            except:
                pass

        feature_arr = []
        for element in depends_results['cells']:
            #print(array)
            array = [0] * (len(feature_list) + 2)
            values = dict(element['values'])
            #print(element)
            array[0] = element['src']
            array[1] = element['dest']
            for feature in feature_list:
                try:
                    value = values[feature]
                    array[feature_list[feature]] = value
                except:
                    pass
            #print(array)
            feature_arr.append(array)

        feature_df = pd.DataFrame(feature_arr)
        col_names = ['src', 'dest']
        for element in feature_list:
            col_names.append(element)
        feature_df.columns = col_names

        feature_df['sum'] = feature_df.sum(axis=1) - feature_df['src'] - feature_df['dest']
        G = nx.Graph()
        for index, row in feature_df.iterrows():
            G.add_edge(row['src'], row['dest'], weight=row['sum'])

        adj_mat = nx.adjacency_matrix(G)
        adj_mat_df = pd.DataFrame(adj_mat.todense())
        np.fill_diagonal(adj_mat_df.values, adj_mat_df.values.max())
        x = adj_mat_df.values
        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(x)
        adj_mat_df = pd.DataFrame(x_scaled)
        
        
        n_cluster_divisible_arr = [5,6,7,8,9,10,15,20,25]
        affinity_arr = ['euclidean', 'l1', 'l2', 'manhattan', 'cosine']
        linkage_arr = ['complete', 'average', 'single']
        
        n_cluster_divisible_arr = [5, 15]
        affinity_arr = ['euclidean']
        linkage_arr = ['complete']
        
        for user_n_cluster in n_cluster_divisible_arr:
            for user_affinity in affinity_arr:
                for user_linkage in linkage_arr:
        
                    cluster = AgglomerativeClustering(n_clusters=(len(cluster_tree)//user_n_cluster), affinity=user_affinity, linkage=user_linkage)
                    cluster_result = cluster.fit_predict(adj_mat_df)
                    
                    filename_a = 'C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + str(user_n_cluster) + '_' +str(user_affinity) + '_' + str(user_linkage) + '_a.rsf'
                    filename_b = 'C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + str(user_n_cluster) + '_' +str(user_affinity) + '_' + str(user_linkage) + '_b.rsf'
                    
                    with open(filename_a, 'w') as f:
                        for key, value in cluster_tree.items():
                            f.write('contain ' + str(value).replace(" ",'') + ' ' + str(key).replace(' ','') + '\n')

                    arr_b_rsf = []
                    with open(filename_b, 'w') as f:
                        len_b_rsf = 0
                        duplicate_array = []
                        for i in range(len(cluster_result)):
                            try:
                                subject = var_df[var_df['index_val'] == i]['name'].values[0]
                                #print(subject)
                                #print(subject in cluster_tree)
                                if subject in cluster_tree and subject not in duplicate_array:
                                    #print(element)
                                    duplicate_array.append(subject)
                                    string = "contain " + str(cluster_result[i]) + " " + subject + "\n"
                                    arr_b_rsf.append(subject)
                                    len_b_rsf += 1
                                    f.write(string)
                                else:
                                    #print(subject)
                                    pass
                            except:
                                pass
                    f.close()

                    len_a_rsf = len(cluster_tree)

                    if len_b_rsf > len_a_rsf:
                        rsf_initial_counter = len_a_rsf
                        to_be_added = list(set(arr_a_rsf) - set(arr_b_rsf))
                        with open(filename_a,'a+') as f:

                            for item in to_be_added:
                                string = 'contain ' + str(rsf_initial_counter) + " " + item + '\n'
                                rsf_initial_counter += 1
                                f.write(string)
                        f.close()

                    elif len_a_rsf > len_b_rsf:
            
                        rsf_initial_counter = len_b_rsf
                        to_be_added = list(set(arr_a_rsf) - set(arr_b_rsf))
                        with open(filename_b,'a+') as f:

                            for item in to_be_added:
                                string = 'contain ' + str(rsf_initial_counter) + " " + item + '\n'
                                rsf_initial_counter += 1
                                f.write(string)
                        f.close()
                        
                    command = 'cd C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1 & ' + 'java MoJo ' + filename_a + ' ' + filename_b + ' >> ' + project_name + '/' + project_name + '_results.txt'
                    #print(command)
                    os.system(command)
                 
                        
            
        f.close()
    

C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/apache-log4j/apache-log4j_8.2.0.json
C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/apache-log4j/apache-log4j_8.1.1.json
C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/apache-log4j/apache-log4j_8.1.0.json
C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/apache-log4j/apache-log4j_8.0.0.json
C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/apache-log4j/apache-log4j_7.7.2.json
C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/apache-log4j/apache-log4j_7.7.1.json
C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/apache-log4j/apache-log4j_7.7.0.json
C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/apache-log4j/apache-log4j_7.6.0.json
C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/apache-log4j/apache-log4j_7.5.0.json
C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/apache-log4j/apache-log4j_7.4.0.json


In [11]:
"""
counter = 10
for row in current_project.iterrows():
    counter -= 1
    if counter < 0:
        break
    project_rank = row[1]['rank']
    #print(project_rank)
    depends_dir = 'C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/' + project_name + '/' + project_name + '_' + row[1]['version_name'] + '.json'
    #print(depends_dir)

    with open(depends_dir) as f:
        depends_results = json.load(f)
        index  = 0
        var_array = []
        for value in depends_results['variables']:
            var_array.append([index, value.split('\\')[-1]])
            #print(index, value)
            index += 1

        var_df = pd.DataFrame(var_array)
        var_df.columns = ['index_val', 'name']

        feature_list = {}
        feature_index = 2
        for element in depends_results['cells']:
            #print(element)
            try:
                for a in element['values']:
                    if a not in feature_list:
                        feature_list[a] = feature_index
                        feature_index += 1
                    #print(a['Call'])


            except:
                pass

        feature_arr = []
        for element in depends_results['cells']:
            #print(array)
            array = [0] * (len(feature_list) + 2)
            values = dict(element['values'])
            #print(element)
            array[0] = element['src']
            array[1] = element['dest']
            for feature in feature_list:
                try:
                    value = values[feature]
                    array[feature_list[feature]] = value
                except:
                    pass
            #print(array)
            feature_arr.append(array)

        feature_df = pd.DataFrame(feature_arr)
        col_names = ['src', 'dest']
        for element in feature_list:
            col_names.append(element)
        feature_df.columns = col_names

        feature_df['sum'] = feature_df.sum(axis=1) - feature_df['src'] - feature_df['dest']
        G = nx.Graph()
        for index, row in feature_df.iterrows():
            G.add_edge(row['src'], row['dest'], weight=row['sum'])

        adj_mat = nx.adjacency_matrix(G)
        adj_mat_df = pd.DataFrame(adj_mat.todense())
        np.fill_diagonal(adj_mat_df.values, adj_mat_df.values.max())
        x = adj_mat_df.values
        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(x)
        adj_mat_df = pd.DataFrame(x_scaled)
        
        
        n_cluster_divisible_arr = [5,6,7,8,9,10,11,12,13,14,15]
        affinity_arr = ['euclidean', 'l1', 'l2', 'manhattan', 'cosine']
        linkage_arr = ['complete', 'average', 'single']
        
        for user_n_cluster in n_cluster_divisible_arr:
            for user_affinity in affinity_arr:
                for user_linkage in linkage_arr:
        
                    cluster = AgglomerativeClustering(n_clusters=user_n_cluster, affinity=user_affinity, linkage=user_linkage)
                    cluster_result = cluster.fit_predict(adj_mat_df)
                    
                    filename_a = 'C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + str(user_n_cluster) + '_' +str(user_affinity) + '_' + str(user_linkage) + '_a.rsf'
                    filename_b = 'C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + str(user_n_cluster) + '_' +str(user_affinity) + '_' + str(user_linkage) + '_b.rsf'
                    with open('test_b.rsf', 'w') as f:
                        len_b_rsf = 0
                        duplicate_array = []
                        for i in range(len(cluster_result)):
                            try:
                                subject = var_df[var_df['index_val'] == i]['name'].values[0]
                                #print(subject)
                                #print(subject in cluster_tree)
                                if subject in cluster_tree and subject not in duplicate_array:
                                    #print(element)
                                    duplicate_array.append(subject)
                                    string = "contain " + str(cluster_result[i]) + " " + subject + "\n"
                                    arr_b_rsf.append(subject)
                                    len_b_rsf += 1
                                    f.write(string)
                                else:
                                    print(subject)
                            except:
                                pass
                    f.close()

                    len_a_rsf = len(cluster_tree)

                    if len_b_rsf > len_a_rsf:
                        rsf_initial_counter = len_a_rsf
                        to_be_added = list(set(arr_a_rsf) - set(arr_b_rsf))
                        with open('test_a.rsf','a+') as f:

                            for item in to_be_added:
                                string = 'contain ' + str(rsf_initial_counter) + " " + item + '\n'
                                rsf_initial_counter += 1
                                f.write(string)
                        f.close()

                    elif len_a_rsf > len_b_rsf:
                        print('here')
                        rsf_initial_counter = len_b_rsf
                        to_be_added = list(set(arr_a_rsf) - set(arr_b_rsf))
                        with open('test_b.rsf','a+') as f:

                            for item in to_be_added:
                                string = 'contain ' + str(rsf_initial_counter) + " " + item + '\n'
                                rsf_initial_counter += 1
                                f.write(string)
                        f.close()
        
        
        
        f.close()
        
        
        
        
"""    
    
    

'\ncounter = 10\nfor row in current_project.iterrows():\n    counter -= 1\n    if counter < 0:\n        break\n    project_rank = row[1][\'rank\']\n    #print(project_rank)\n    depends_dir = \'C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/\' + project_name + \'/\' + project_name + \'_\' + row[1][\'version_name\'] + \'.json\'\n    #print(depends_dir)\n\n    with open(depends_dir) as f:\n        depends_results = json.load(f)\n        index  = 0\n        var_array = []\n        for value in depends_results[\'variables\']:\n            var_array.append([index, value.split(\'\\\')[-1]])\n            #print(index, value)\n            index += 1\n\n        var_df = pd.DataFrame(var_array)\n        var_df.columns = [\'index_val\', \'name\']\n\n        feature_list = {}\n        feature_index = 2\n        for element in depends_results[\'cells\']:\n            #print(element)\n            try:\n                for a in element[\'values\']:\n                    if a not in featur

In [ ]:

index  = 0
var_array = []
for value in depends_results['variables']:
    var_array.append([index, value.split('\\')[-1]])
    #print(index, value)
    index += 1

var_df = pd.DataFrame(var_array)
var_df.columns = ['index_val', 'name']

feature_list = {}
feature_index = 2
for element in depends_results['cells']:
    #print(element)
    try:
        for a in element['values']:
            if a not in feature_list:
                feature_list[a] = feature_index
                feature_index += 1
            #print(a['Call'])


    except:
        pass

feature_arr = []
for element in depends_results['cells']:
    #print(array)
    array = [0] * (len(feature_list) + 2)
    values = dict(element['values'])
    #print(element)
    array[0] = element['src']
    array[1] = element['dest']
    for feature in feature_list:
        try:
            value = values[feature]
            array[feature_list[feature]] = value
        except:
            pass
    #print(array)
    feature_arr.append(array)

feature_df = pd.DataFrame(feature_arr)
col_names = ['src', 'dest']
for element in feature_list:
    col_names.append(element)
feature_df.columns = col_names

feature_df['sum'] = feature_df.sum(axis=1) - feature_df['src'] - feature_df['dest']
G = nx.Graph()
for index, row in feature_df.iterrows():
    G.add_edge(row['src'], row['dest'], weight=row['sum'])

adj_mat = nx.adjacency_matrix(G)
adj_mat_df = pd.DataFrame(adj_mat.todense())
np.fill_diagonal(adj_mat_df.values, adj_mat_df.values.max())
x = adj_mat_df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
adj_mat_df = pd.DataFrame(x_scaled)

In [ ]:
var_df.head()
#for row in var_df.iterrows():
    #print(row[1]['name'])

In [ ]:
print(len(var_df))

In [ ]:
cluster = AgglomerativeClustering(n_clusters=30, affinity='euclidean', linkage='complete')
cluster_result = cluster.fit_predict(adj_mat_df)

In [ ]:
print(len(cluster_result))
cluster_result

In [ ]:
cluster_tree

In [12]:
8//3

2

In [ ]:
with open('test_b.rsf', 'w') as f:
    len_b_rsf = 0
    duplicate_array = []
    for i in range(len(cluster_result)):
        try:
            subject = var_df[var_df['index_val'] == i]['name'].values[0]
            #print(subject)
            #print(subject in cluster_tree)
            if subject in cluster_tree and subject not in duplicate_array:
                #print(element)
                duplicate_array.append(subject)
                string = "contain " + str(cluster_result[i]) + " " + subject + "\n"
                arr_b_rsf.append(subject)
                len_b_rsf += 1
                f.write(string)
            else:
                print(subject)
        except:
            pass
    f.close()

len_a_rsf = len(cluster_tree)

if len_b_rsf > len_a_rsf:
    rsf_initial_counter = len_a_rsf
    to_be_added = list(set(arr_a_rsf) - set(arr_b_rsf))
    with open('test_a.rsf','a+') as f:
        
        for item in to_be_added:
            string = 'contain ' + str(rsf_initial_counter) + " " + item + '\n'
            rsf_initial_counter += 1
            f.write(string)
    f.close()
        
elif len_a_rsf > len_b_rsf:
    print('here')
    rsf_initial_counter = len_b_rsf
    to_be_added = list(set(arr_a_rsf) - set(arr_b_rsf))
    with open('test_b.rsf','a+') as f:
        
        for item in to_be_added:
            string = 'contain ' + str(rsf_initial_counter) + " " + item + '\n'
            rsf_initial_counter += 1
            f.write(string)
    f.close()

In [ ]:
'AppenderTable.java' in cluster_tree

In [ ]:
len(cluster_tree)

In [ ]:
arr_a_rsf = []
arr_b_rsf = []


In [ ]:

for key in cluster_tree:
    arr_a_rsf.append(key)
    
    


In [ ]:
list(set(test_arr_1) - set(test_arr_2))

In [ ]:
test_arr_2